In [22]:
print("Experiments commit 80ac88c")

Experiments commit 80ac88c


In [23]:
# Librerias

import pandas as pd
import numpy as np
import re
import nltk
import tensorflow
import tensorflow.keras
import tensorflow.keras.backend
import tensorflow.keras.models as models
from nltk.corpus import stopwords
import random as rn
import os

from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM, Bidirectional, concatenate
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

In [24]:
# Funciones utiles
import sys
import os
import pickle

#pickle_path = "/datos/ecampillo/jupyter/dl-notebooks/pickles"
pickle_path = "/datos/ecampillo/jupyter/dl-notebooks/newensemble/early-risk-ensemble/pickles"

def logger(message, debug_file="log.txt"):
    print(message)
    original_stdout = sys.stdout # Save a reference to the original standard output
    with open(debug_file, 'a') as f:
        sys.stdout = f # Change the standard output to the file we created.
        print(message)
        sys.stdout = original_stdout # Reset the standard output to its original value
        
def save_pickle(filepath, filename, data):
    if not os.path.exists(filepath):
        os.makedirs(filepath)
    file = os.path.join(filepath, filename)
    with open(file, 'wb') as data_file:
        pickle.dump(data, data_file)
        
def load_pickle(filepath, filename):
    file = os.path.join(filepath, filename)
    with open(file, 'rb') as data_file:
        data = pickle.load(data_file)
    return data

In [25]:
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)
tensorflow.random.set_seed(42) 

logger("Initialized numpy random and tensorflow random seed at 42")

Initialized numpy random and tensorflow random seed at 42


In [26]:
#train_users = load_pickle(pickle_path, "train_users.pkl")
#test_users = load_pickle(pickle_path, "test_users.pkl")
X_train = load_pickle(pickle_path, "X_train_over.pkl")
X_test = load_pickle(pickle_path, "X_test.pkl")
y_train = load_pickle(pickle_path, "y_train_over.pkl")
y_test = load_pickle(pickle_path, "y_test.pkl")

In [28]:
feats_train = load_pickle(pickle_path, "feats_train_original.pkl")
feats_test = load_pickle(pickle_path, "feats_test_original.pkl")

### Normalize

In [29]:
normalize_exceptions = ['char_count', 'word_density']

def normalize_features(feats):
    text_length = feats["char_count"]
    
    norm_feats = pd.DataFrame()
    for feature in feats.columns:
        if feature not in normalize_exceptions:
            norm_feats[feature] = feats[feature] / text_length
            
    return norm_feats

### Discretize

In [30]:
from sklearn.preprocessing import KBinsDiscretizer

def discretize_features(train_feats, test_feats, size=10, strategy='kmeans', encode='onehot-dense'):
    est = KBinsDiscretizer(n_bins=size, encode=encode, strategy=strategy)
    train = est.fit_transform(train_feats)
    test = est.transform(test_feats)

    return train, test

### Functions to select features

In [31]:
def select_features(exclude_feats=[], normalize=False, discretize=False, discretize_size=10):
    feats_train_ret = feats_train.copy()
    feats_test_ret = feats_test.copy()
    
    for feat in exclude_feats:
        feats_train_ret.drop(feat, inplace=True, axis=1)
        feats_test_ret.drop(feat, inplace=True, axis=1)
    
    if normalize:
        feats_train_ret = normalize_features(feats_train_ret)
        feats_test_ret = normalize_features(feats_test_ret)
        
    if discretize:
        feats_train_ret, feats_test_ret = discretize_features(feats_train_ret, feats_test_ret, size=discretize_size)
    else:
        feats_train_ret = feats_train_ret.values
        feats_test_ret = feats_test_ret.values
    
    return feats_train_ret, feats_test_ret

### DL preprocc

In [32]:
tokenizer = Tokenizer(num_words=50000) # 5000
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Padding

# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 50000  # podria ser la que quisieramos  # antes tenia 10000, voy a probar con 50000

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [33]:
# Loading the GloVe embeddings

from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('/datos/erisk/deep-learning/embeddings/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [34]:
# Creating an embedding matrix

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [35]:
save_pickle(pickle_path, "embedding_matrix.pkl", embedding_matrix)

In [11]:
embedding_matrix = load_pickle(pickle_path, "embedding_matrix.pkl")

In [36]:
def define_cnn_model(loc_input_len):
    tensorflow.keras.backend.clear_session()
    meta_input = Input(shape=(loc_input_len,))
    nlp_input = Input(shape=(maxlen,))
    emb = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)(nlp_input)
    nlp_out = Conv1D(64, 5, activation='relu')(emb)
    max_pool = GlobalMaxPooling1D()(nlp_out)
    concat = concatenate([max_pool, meta_input])
    classifier = Dense(32, activation='relu')(concat)
    output = Dense(1, activation='sigmoid')(classifier)
        
    return Model(inputs=[nlp_input, meta_input], outputs=[output])

In [37]:
def define_lstm_model(loc_input_len):
    tensorflow.keras.backend.clear_session()
    meta_input = Input(shape=(loc_input_len,))
    nlp_input = Input(shape=(maxlen,)) 
    emb = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)(nlp_input)
    nlp_out = Bidirectional(LSTM(128))(emb)
    concat = concatenate([nlp_out, meta_input])
    classifier = Dense(32, activation='relu')(concat)
    output = Dense(1, activation='sigmoid')(classifier)
    
    return Model(inputs=[nlp_input , meta_input], outputs=[output])

In [38]:
def train_model(model_to_train, X_train, feats_train, y_train):
    history = model_to_train.fit([X_train, feats_train], y_train, batch_size=2, epochs=10, verbose=1, validation_split=0.2, shuffle=True)
    return history

### Eval functions

In [39]:
def evaluate_model(model_eval, X_test, feats_test, y_test):
    score = model_eval.evaluate([X_test, feats_test], y_test, verbose=1)
    logger("Test Score: {}".format(score[0]))
    logger("Test Accuracy: {}".format(score[1]))

    from sklearn.metrics import classification_report, confusion_matrix
    import numpy as np

    y_pred = model_eval.predict([X_test, feats_test], batch_size=2, verbose=1)
    if y_pred.shape[-1] > 1:
        y_pred_label = y_pred.argmax(axis=-1)
    else:
        print("Entered here")
        y_pred_label = (y_pred > 0.5).astype('int32')

    from sklearn.metrics import classification_report, confusion_matrix

    logger(classification_report(y_test, y_pred_label))
    logger(confusion_matrix(y_test, y_pred_label))

## CNN Experiments

In [40]:
model = ""
if model is not None:
    print("Deleting model")
    del(model)

Deleting model


In [41]:
logger("Experiment 1: CNN, First person, sentiment analysis y nssi dictionary sin normalizar ni discretizar")

# selecting features
tensorflow.keras.backend.clear_session()
train_feats_new, test_feats_new = select_features()
logger(len(train_feats_new[1,]))
model = define_cnn_model(len(train_feats_new[1,]))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
logger("Evaluating before training")
evaluate_model(model, X_test, test_feats_new, y_test)
logger("Training")
history = model.fit([X_train, train_feats_new], y_train, batch_size=2, epochs=10, verbose=1, validation_split=0.2, shuffle=True)
logger("Evaluating")
evaluate_model(model, X_test, test_feats_new, y_test)
del(model)

Experiment 1: CNN, First person, sentiment analysis y nssi dictionary sin normalizar ni discretizar
8
Evaluating before training
14/14 [==============================] - 12s 820ms/step - loss: 24.8152 - acc: 0.7541
Test Score: 24.81515884399414
Test Accuracy: 0.7541370987892151
212/212 [==============================] - 8s 35ms/step
Entered here
              precision    recall  f1-score   support

           0       0.75      1.00      0.86       319
           1       0.00      0.00      0.00       104

    accuracy                           0.75       423
   macro avg       0.38      0.50      0.43       423
weighted avg       0.57      0.75      0.65       423

[[319   0]
 [104   0]]
Training


/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
239/239 [==============================] - 65s 269ms/step - loss: 60.3041 - acc: 0.5837 - val_loss: 205.0491 - val_acc: 0.0083
Epoch 2/10
239/239 [==============================] - 88s 369ms/step - loss: 35.8358 - acc: 0.6569 - val_loss: 21.0317 - val_acc: 0.6250
Epoch 3/10
239/239 [==============================] - 77s 320ms/step - loss: 30.1783 - acc: 0.6987 - val_loss: 58.4560 - val_acc: 0.2333
Epoch 4/10
239/239 [==============================] - 75s 316ms/step - loss: 32.1477 - acc: 0.7113 - val_loss: 2.6565e-19 - val_acc: 1.0000
Epoch 5/10
239/239 [==============================] - 69s 289ms/step - loss: 50.0798 - acc: 0.6820 - val_loss: 11.4584 - val_acc: 0.8083
Epoch 6/10
239/239 [==============================] - 73s 304ms/step - loss: 66.7952 - acc: 0.7238 - val_loss: 551.0635 - val_acc: 0.2417
Epoch 7/10
239/239 [==============================] - 73s 305ms/step - loss: 54.0123 - acc: 0.7782 - val_loss: 4.2690 - val_acc: 0.9167
Epoch 8/10
239/239 [=================

In [ ]:
logger("Experiment 1: CNN, First person, sentiment analysis y nssi dictionary sin normalizar ni discretizar")

# selecting features

train_feats_new, test_feats_new = select_features()
logger(len(train_feats_new[1,]))
model = define_cnn_model(len(train_feats_new[1,]))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
logger("Evaluating before training")
evaluate_model(model, X_test, test_feats_new, y_test)
logger("Training")
history = train_model(model, X_train, train_feats_new, y_train)
logger("Evaluating")
evaluate_model(model, X_test, test_feats_new, y_test)
del(model)

In [ ]:
logger("Experiment 2: CNN, First person, sentiment analysis y nssi dictionary con normalizar ni discretizar")

# selecting features
train_feats_new, test_feats_new = select_features(normalize=True)
logger(len(train_feats_new[1,]))
model = define_cnn_model(len(train_feats_new[1,]))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
logger(model.summary())
logger("Evaluating before training")
evaluate_model(model, X_test, test_feats_new, y_test)
logger("Training")
history = train_model(model, X_train, train_feats_new, y_train)
logger("Evaluating")
evaluate_model(model, X_test, test_feats_new, y_test)

In [ ]:
logger("Experiment 3: CNN, First person, sentiment analysis y nssi dictionary sin normalizar con discretizar")

# selecting features
train_feats_new, test_feats_new = select_features(discretize=True)
model = define_cnn_model(len(train_feats_new[1,]))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
logger("Evaluating before training")
evaluate_model(model, X_test, test_feats_new, y_test)
logger("Training")
history = train_model(model, X_train, train_feats_new, y_train)
logger("Evaluating")
evaluate_model(model, X_test, test_feats_new, y_test)

In [ ]:
logger("Experiment 4: CNN, First person, sentiment analysis y nssi dictionary sin normalizar con discretizar")

# selecting features
tensorflow.keras.backend.clear_session()
train_feats_new, test_feats_new = select_features(discretize=True)
model = define_cnn_model(len(train_feats_new[1,]))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
logger("Evaluating before training")
evaluate_model(model, X_test, test_feats_new, y_test)
logger("Training")
history = train_model(model, X_train, train_feats_new, y_train)
logger("Evaluating")
evaluate_model(model, X_test, test_feats_new, y_test)

In [ ]:
logger("Experiment 5: CNN, First person, sentiment analysis y nssi dictionary con normalizar con discretizar")

# selecting features
train_feats_new, test_feats_new = select_features(normalize=True, discretize=True)
model = define_cnn_model(len(train_feats_new[1,]))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
logger("Evaluating before training")
evaluate_model(model, X_test, test_feats_new, y_test)
logger("Training")
history = train_model(model, X_train, train_feats_new, y_train)
logger("Evaluating")
evaluate_model(model, X_test, test_feats_new, y_test)

#### Solo first person

In [42]:
logger("Experiment 6.0: CNN, First person sin normalizar sin discretizar")

# selecting features
train_feats_new, test_feats_new = select_features(exclude_feats=["sentiment", "methods", 
                                                                 "terms", "instruments", "reasons"])
logger(len(train_feats_new[1,]))
model = define_cnn_model(train_feats_new[1,].shape[0])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
logger("Evaluating before training")
evaluate_model(model, X_test, test_feats_new, y_test)
logger("Training")
#history = train_model(model, X_train, train_feats_new, y_train)
history = model.fit([X_train, train_feats_new], y_train, batch_size=2, epochs=10, verbose=1, validation_split=0.2, shuffle=True)
logger("Evaluating")
evaluate_model(model, X_test, test_feats_new, y_test)

Experiment 6.0: CNN, First person sin normalizar sin discretizar
3
Evaluating before training
14/14 [==============================] - 7s 470ms/step - loss: 194.6896 - acc: 0.7541
Test Score: 194.68960571289062
Test Accuracy: 0.7541370987892151
212/212 [==============================] - 7s 33ms/step
Entered here
              precision    recall  f1-score   support

           0       0.75      1.00      0.86       319
           1       0.00      0.00      0.00       104

    accuracy                           0.75       423
   macro avg       0.38      0.50      0.43       423
weighted avg       0.57      0.75      0.65       423

[[319   0]
 [104   0]]
Training
Epoch 1/10


/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


239/239 [==============================] - 59s 243ms/step - loss: 58.2047 - acc: 0.5816 - val_loss: 371.6874 - val_acc: 0.0000e+00
Epoch 2/10
239/239 [==============================] - 66s 278ms/step - loss: 111.5344 - acc: 0.5983 - val_loss: 278.9637 - val_acc: 0.0083
Epoch 3/10
239/239 [==============================] - 68s 287ms/step - loss: 86.1810 - acc: 0.6234 - val_loss: 1.0176e-09 - val_acc: 1.0000
Epoch 4/10
239/239 [==============================] - 67s 279ms/step - loss: 40.7051 - acc: 0.7071 - val_loss: 41.3716 - val_acc: 0.6167
Epoch 5/10
239/239 [==============================] - 67s 280ms/step - loss: 33.9198 - acc: 0.7741 - val_loss: 98.1848 - val_acc: 0.1917
Epoch 6/10
239/239 [==============================] - 67s 280ms/step - loss: 94.5895 - acc: 0.7176 - val_loss: 637.7128 - val_acc: 0.2417
Epoch 7/10
239/239 [==============================] - 68s 285ms/step - loss: 72.6668 - acc: 0.7636 - val_loss: 2.9642e-08 - val_acc: 1.0000
Epoch 8/10
239/239 [==================

In [ ]:
logger("Experiment 6: CNN, First person sin normalizar con discretizar")

# selecting features
train_feats_new, test_feats_new = select_features(exclude_feats=["sentiment", "methods", 
                                                                 "terms", "instruments", "reasons"],
                                                  discretize=True)
model = define_cnn_model(train_feats_new[1,].shape[0])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
logger("Evaluating before training")
evaluate_model(model, X_test, test_feats_new, y_test)
logger("Training")
history = train_model(model, X_train, train_feats_new, y_train)
logger("Evaluating")
evaluate_model(model, X_test, test_feats_new, y_test)

In [ ]:
logger("Experiment 7: CNN, First person con normalizar con discretizar")

# selecting features
train_feats_new, test_feats_new = select_features(exclude_feats=["sentiment", "methods", 
                                                                 "terms", "instruments", "reasons"],
                                                  discretize=True, normalize=True)
model = define_cnn_model(train_feats_new[1,].shape[0])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
logger("Evaluating before training")
evaluate_model(model, X_test, test_feats_new, y_test)
logger("Training")
history = train_model(model, X_train, train_feats_new, y_train)
logger("Evaluating")
evaluate_model(model, X_test, test_feats_new, y_test)

#### First person y sentiment

In [43]:
logger("Experiment 8: CNN, First person y sentiment sin normalizar con discretizar")

# selecting features
train_feats_new, test_feats_new = select_features(exclude_feats=["methods", "terms", "instruments", "reasons"],
                                                  discretize=True)
model = define_cnn_model(train_feats_new[1,].shape[0])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
logger("Evaluating before training")
evaluate_model(model, X_test, test_feats_new, y_test)
logger("Training")
history = train_model(model, X_train, train_feats_new, y_train)
logger("Evaluating")
evaluate_model(model, X_test, test_feats_new, y_test)

Experiment 8: CNN, First person y sentiment sin normalizar con discretizar
Evaluating before training
14/14 [==============================] - 7s 447ms/step - loss: 0.5847 - acc: 0.7541
Test Score: 0.5847447514533997
Test Accuracy: 0.7541370987892151
212/212 [==============================] - 7s 32ms/step
Entered here
              precision    recall  f1-score   support

           0       0.75      1.00      0.86       319
           1       0.00      0.00      0.00       104

    accuracy                           0.75       423
   macro avg       0.38      0.50      0.43       423
weighted avg       0.57      0.75      0.65       423

[[319   0]
 [104   0]]
Training


/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
239/239 [==============================] - 58s 240ms/step - loss: 0.4088 - acc: 0.8243 - val_loss: 0.5377 - val_acc: 0.7917
Epoch 2/10
239/239 [==============================] - 69s 288ms/step - loss: 0.0917 - acc: 0.9812 - val_loss: 0.0143 - val_acc: 1.0000
Epoch 3/10
239/239 [==============================] - 71s 297ms/step - loss: 0.0183 - acc: 1.0000 - val_loss: 0.0176 - val_acc: 1.0000
Epoch 4/10
239/239 [==============================] - 70s 293ms/step - loss: 0.0066 - acc: 1.0000 - val_loss: 0.0063 - val_acc: 1.0000
Epoch 5/10
239/239 [==============================] - 68s 282ms/step - loss: 0.0035 - acc: 1.0000 - val_loss: 0.0037 - val_acc: 1.0000
Epoch 6/10
239/239 [==============================] - 71s 297ms/step - loss: 0.0022 - acc: 1.0000 - val_loss: 0.0023 - val_acc: 1.0000
Epoch 7/10
239/239 [==============================] - 77s 323ms/step - loss: 0.0014 - acc: 1.0000 - val_loss: 0.0019 - val_acc: 1.0000
Epoch 8/10
239/239 [==============================] - 7

In [ ]:
logger("Experiment 8: CNN, First person y sentiment con normalizar sin discretizar")

# selecting features
train_feats_new, test_feats_new = select_features(exclude_feats=["methods", "terms", "instruments", "reasons"],
                                                  normalize=True)
model = define_cnn_model(train_feats_new[1,].shape[0])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
logger("Evaluating before training")
evaluate_model(model, X_test, test_feats_new, y_test)
logger("Training")
history = train_model(model, X_train, train_feats_new, y_train)
logger("Evaluating")
evaluate_model(model, X_test, test_feats_new, y_test)

In [ ]:
logger("Experiment 9: CNN, First person y sentiment con normalizar con discretizar")

# selecting features
train_feats_new, test_feats_new = select_features(exclude_feats=["methods", "terms", "instruments", "reasons"],
                                                  normalize=True, discretize=True)
model = define_cnn_model(train_feats_new[1,].shape[0])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
logger("Evaluating before training")
evaluate_model(model, X_test, test_feats_new, y_test)
logger("Training")
history = train_model(model, X_train, train_feats_new, y_train)
logger("Evaluating")
evaluate_model(model, X_test, test_feats_new, y_test)